In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor

In [ ]:
train = pd.read_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\train.csv')
test = pd.read_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\test.csv')

In [ ]:
train = train.iloc[:,1:train.shape[1]]
train = train.drop('dropoff_datetime',1)
test = test.iloc[:,1:test.shape[1]]

In [ ]:
train['ptime'] = train['pickup_datetime'].astype(datetime)
print(train['ptime'][:5])
test['ptime'] = test['pickup_datetime'].astype(datetime)
print(test['ptime'][:5])

In [ ]:
def h_m_features(ser):
    #print(ser[:5])
    hours = []
    mini = []
    #print(ser[2])
    for i in range(len(ser)):
        #print(i)
        tim = str(ser[i][11:])
        tim = tim.split(':')
        tim = tim[:2]
        tim = [int(x) for x in tim]
        #print(tim,i)
        hours.append(tim[0])
        mini.append(tim[1])
    return hours,mini

In [ ]:
hours1, mini1 = h_m_features(train['ptime'])
train['phours'] = hours1
train['mini1'] = mini1
hours1, mini1 = h_m_features(test['ptime'])
test['phours'] = hours1
test['mini1'] = mini1

In [ ]:
train = train.drop('ptime',1)
test = test.drop('ptime',1)
train = train.drop('pickup_datetime',1)
test = test.drop('pickup_datetime',1)

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [ ]:
for dataset in [train,test]:
    tripDistance = []
    for j in range(dataset.shape[0]):
        long1 = dataset['pickup_longitude'][j]
        lati1 = dataset['pickup_latitude'][j]
        long2 = dataset['dropoff_longitude'][j]
        lati2 = dataset['dropoff_latitude'][j]
        tripDistance.append(haversine(long1,lati1,long2,lati2))
    dataset['trip_distance'] = tripDistance
print(train['trip_distance'][:5])

In [ ]:
test['trip_distance'].describe()

In [ ]:
train = train.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],1)
test = test.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],1)

In [ ]:
train['store_and_fwd_flag'] = train['store_and_fwd_flag'].map({'N':1,'Y':0}).astype(int)
test['store_and_fwd_flag'] = test['store_and_fwd_flag'].map({'N':1,'Y':0}).astype(int)

In [ ]:
train.to_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\train_cleaned.csv',index=False)
test.to_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\test_cleaned.csv',index=False)

In [ ]:
train = pd.read_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\train_cleaned.csv')
test = pd.read_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\test_cleaned.csv')
out = pd.read_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\sample_submission.csv')
print(train.head())
print(test.head())

In [ ]:
#train1 = train[train['trip_duration']<87000]
train1 = train.copy()

In [ ]:
x = train1.drop('trip_duration',1)
y = train1['trip_duration']
x = preprocessing.scale(x)
#y = preprocessing.scale(y)
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.33,random_state=99)
print(x_train.shape,y_train.shape,x_valid.shape,y_valid.shape,test.shape)
#print(y.describe())

In [ ]:
reg = SGDRegressor(max_iter=1000, verbose=1)
#reg = SVR(kernel='linear')
#reg = MLPRegressor()
#reg = LinearRegression()
#x_train = preprocessing.scale(x_train)
#y_train = preprocessing.scale(y_train)
print(reg)
reg.fit(x_train,y_train)
preds = reg.predict(x_valid)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_valid,preds)
print(mse)
print(pd.DataFrame(preds).describe())
#out['trip_duration'] = preds
#out['trip_duration'][out['trip_duration']<0] = out['trip_duration'].mean()
#print(out['trip_duration'].describe())
#out.to_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\sample_submission3.csv', index=False)

In [ ]:
#reg = SGDRegressor()
print(reg)
#reg = SVR(kernel='linear')
#reg = MLPRegressor()
#reg = LinearRegression()
#x_train = preprocessing.scale(x_train)
#y_train = preprocessing.scale(y_train)
#reg.fit(x_train,y_train)
#preds = reg.predict(x_valid)
#from sklearn.metrics import mean_squared_error
#mse = mean_squared_error(y_valid,preds)
#print(mse)
#print(pd.DataFrame(preds).describe())
test = preprocessing.scale(test)
preds = reg.predict(test)
out['trip_duration'] = preds
out['trip_duration'][out['trip_duration']<0] = out['trip_duration'].mean()
print(out['trip_duration'].describe())
out.to_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\sample_submissionSGD.csv', index=False)

In [ ]:
#reg = SGDRegressor()
reg = SVR(kernel='linear',verbose=100)
#reg = MLPRegressor()
#reg = LinearRegression()
print(reg)
#x_train = preprocessing.scale(x_train)
#y_train = preprocessing.scale(y_train)
reg.fit(x_train,y_train)
#preds = reg.predict(x_valid)
#from sklearn.metrics import mean_squared_error
#mse = mean_squared_error(y_valid,preds)
#print(mse)
#print(pd.DataFrame(preds).describe())
preds = reg.predict(test)
out['trip_duration'] = preds
out['trip_duration'][out['trip_duration']<0] = out['trip_duration'].mean()
print(out['trip_duration'].describe())
out.to_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\sample_submissionSVR.csv', index=False)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
#reg = SGDRegressor()
#reg = SVR(kernel='linear')
reg = MLPRegressor()
#reg = LinearRegression()
print(reg)
#x_train = preprocessing.scale(x_train)
#y_train = preprocessing.scale(y_train)
reg.fit(x_train,y_train)
#preds = reg.predict(x_valid)
#from sklearn.metrics import mean_squared_error
#mse = mean_squared_error(y_valid,preds)
#print(mse)
#print(pd.DataFrame(preds).describe())
preds = reg.predict(test)
out['trip_duration'] = preds
out['trip_duration'][out['trip_duration']<0] = out['trip_duration'].mean()
print(out['trip_duration'].describe())
out.to_csv('C:\\Users\\venka\\Desktop\\Kaggle\\NY city Taxi\\sample_submissionMLP.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train1['trip_duration']/60)
plt.show()